# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


/Users/henriquebueno/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/models-master/research/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/henriquebueno/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/henriquebueno/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/henriquebueno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/henriquebueno/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/henriquebuen

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
#alteraçoes henrique aqui

# What model to download.
#MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_NAME = 'celular_inference_graph'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
#PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
PATH_TO_LABELS = os.path.join('training', 'object-detection.pbtxt')

#NUM_CLASSES = 90
NUM_CLASSES = 1

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
#print(categories)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [7]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [8]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#PATH_TO_TEST_IMAGES_DIR = 'test_images'
PATH_TO_TEST_IMAGES_DIR = '/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data'
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}.jpg'.format(i)) for i in [0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,20,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,57,58,59,60,61,62,63,64,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134] ]
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}.jpg'.format(i)) for i in [0,1] ]

# Size, in inches, of the output images.
#IMAGE_SIZE = (12, 8)
#IMAGE_SIZE = (490, 326)
IMAGE_SIZE = (5, 5)

In [9]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [10]:
import math
import numpy as np

def temTelefone(detection_boxes1, detection_classes1, detection_scores1, fileName):

    xEstimado = -1
    yEstimado = -1
    
    #so estou considerando uma classe entao pegarei o box com maior score
    indice = np.argmax(detection_scores1)
    box = detection_boxes1[indice]

    #im_width=490
    #im_height=326
    
    #print("(left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)")
    #print("box: " + str(box))
    
    ymin = box[0]
    xmin = box[1]
    ymax = box[2]
    xmax = box[3]
    #print("(xmin, xmax, ymin, ymax)")
    #print(xmin, xmax, ymin, ymax)
    
    xEstimado=(xmax+xmin)/2
    yEstimado=(ymax+ymin)/2
    #print("(xEstimado, yEstimado)")
    #print(xEstimado,yEstimado)
    
    #left = box[0]
    #right = box[1]
    #top = box[2]
    #bottom = box[3]
    
    #xmin=left/im_width
    #xmax=right/im_width
    #ymin=top/im_height
    #ymax=bottom/im_height
    #print(xmin, xmax, ymin, ymax)
    
    #for pos in range(len(detection_classes1)):
        #print("-----------classe: " + str(detection_classes1[pos]))
        #if detection_classes1[pos]==77:#classe cell phone
        #if detection_classes1[pos]==1:#classe celular
            #print("temTelefone - Nome do arquivo: " + str(fileName) + "confidence: "+str(detection_scores1[pos])+ "classe: "+str(detection_classes1[pos]))
            #coordenadas = detection_boxes1[pos]
            #print("Posicao do celular encontrado: " + str(coordenadas))
            #ymin = coordenadas[0]
            #xmin = coordenadas[1]
            #ymax = coordenadas[2]
            #xmax = coordenadas[3]
            #print("Y min: " + str(ymin))
            #print("X min: " + str(xmin))
            #print("Y max: " + str(ymax))
            #print("X max: " + str(xmax))
            #xEstimado=(xmin+xmax)/2
            #yEstimado=(ymin+ymax)/2
            #xCorreto0=0.8306 
            #yCorreto0=0.1350

            #TAMBÉM PRECISA ALTERAR A CELULA DETECTION--------------------------

            #Nao achou para a 1.jpg
            #xCorreto1=0.8714
            #yCorreto1=0.1718

            #xCorreto3=0.6857
            #yCorreto3=0.6933

            #print("Ponto: " + str(xEstimado) + " - " + str(yEstimado))
            #print("Score do celular encontrado: " + str(detection_scores1[pos]))
            #dist = math.hypot(xEstimado - xCorreto3, yEstimado - yCorreto3)
            #print("Distancia: " + str(dist))
            
    return xEstimado, yEstimado

In [11]:
#carrega o arquivo com o nome dos arquivos e as coordenadas dos celulares em cada
#arquivo em uma matriz onde cada linha é um registro do arquivo labels.txt e cada
#linha tem 3 valores: NOME_ARQUIVO X Y
#ATENCAO: os 3 valores de cada linha sao armazenados como string

def carregaArquivo(arquivo):

    #abre o arquivo
    file = open(arquivo, "r")

    #le todas as linhas
    linhas=file.readlines()
    
    #fecha arquivo
    file.close()

    return linhas

In [12]:
#Funcao que recebe como entrada um array com os registros de entrada e um
#string com o NOME_ARQ. O retorno é a posicao do arquivo NOME_ARQ em registros.
#Caso ele não encontre o nome, ele retorna -1
def localiza_X_e_Y(linhas, nomeArquivo):
    indiceLinha = -1
    xRetorno = -1
    yRetorno = -1
    
    pos=0
    for linha in linhas:
        temp = linha.split()
        nome=temp[0]
        nome = PATH_TO_TEST_IMAGES_DIR+"/" + nome
        #print(nome)
        if(nome==nomeArquivo):
            indiceLinha = pos
            break
        pos+=1
        
    if(indiceLinha != -1):
        linha = linhas[pos]
        linhaTokenizada = linha.split()
        xRetorno=float(linhaTokenizada[1])
        yRetorno=float(linhaTokenizada[2])
        
    return xRetorno, yRetorno

In [13]:
#Apenas um teste

#arquivo = "/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/labels.txt"
#linhas = carregaArquivo(arquivo)
#x, y = localiza_X_e_Y(linhas, "/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/0.jpg")
#print(x)
#print(y)

In [14]:
import math

def distancia(x1,y1,x2,y2):
     dist = ((x1 - x2)**2 + (y1-y2)**2)**.5 
     return dist

In [15]:
#predicao

arquivo = "/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/labels.txt"
linhas = carregaArquivo(arquivo)
#pos = localizaArquivoEmLinhas(linhas, "129.jpg")
#print(pos)
#linha = linhas[pos]
#linhaTokenizada = linha.split()
#print(linhaTokenizada)

sim = 0
nao = 0
simComDistancia=0

for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  #print("abriu " + str(image.filename))
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  #print("ja rodou inferencia " + str(image.filename))
  # Visualization of the results of a detection.
  #vis_util.visualize_boxes_and_labels_on_image_array(
  #    image_np,
  #    output_dict['detection_boxes'],
  #    output_dict['detection_classes'],
  #    output_dict['detection_scores'],
  #    category_index,
  #    instance_masks=output_dict.get('detection_masks'),
  #    use_normalized_coordinates=True,
  #    line_thickness=3)
  #print("vis_util.visualize_boxes_and_labels_on_image_array " + str(image.filename))
  #plt.figure(figsize=IMAGE_SIZE)
  #print("plt.figure " + str(image.filename))
  #plt.imshow(image_np)
  #print("plt.imshow " + str(image.filename))

  detection_boxes1 = output_dict['detection_boxes']
  detection_classes1 = output_dict['detection_classes']
  detection_scores1 = output_dict['detection_scores']
   
  #print(detection_boxes1)
  #coordenadas = detection_boxes1[pos]
  #print("Posicao do celular encontrado: " + str(coordenadas))
  #ymin = coordenadas[0]
  #xmin = coordenadas[1]
  #ymax = coordenadas[2]
  #xmax = coordenadas[3]
  #print("Y min: " + str(ymin))
  #print("X min: " + str(xmin))
  #print("Y max: " + str(ymax))
  #print("X max: " + str(xmax))
  #xEstimado=(xmin+xmax)/2
  #yEstimado=(ymin+ymax)/2
    
  #(left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
  xEstimado, yEstimado = temTelefone(detection_boxes1, detection_classes1, detection_scores1, image.filename)

  if((xEstimado==-1)and(yEstimado==-1)):
    #print("Não: " + str(image.filename))
    print("NAO")
    nao+=1
  else:
    #pos = localizaArquivoEmLinhas(linhas, image.filename)
    #linha = linhas[pos]
    #linhaTokenizada = linha.split()
    #xCorreto=float(linhaTokenizada[1])
    #yCorreto=float(linhaTokenizada[2])
    #vai consultar o arquivo do professor com as coordenadas para encontrar as coord
    #corretas de image.filename. Atencao: o nome do arquivo tem que estar completo com a pasta
    xCorreto, yCorreto = localiza_X_e_Y(linhas, image.filename)
    
    #dist = math.hypot(xEstimado - xCorreto, yEstimado - yCorreto)    
    dist = distancia(xEstimado,yEstimado,xCorreto,yCorreto)
    if(dist<0.05):
        simComDistancia+=1
    
    #print("Sim: " + str(image.filename) + " x, y= " + str(xEstimado) + "-" + str(yEstimado))
    print("SIM " + str(dist) + " xEst " + str(xEstimado) + " yEst " + str(yEstimado) + " xCorr " + str(xCorreto) + " yCorr " + str(yCorreto) + image_path)
    sim+=1
    
#ainda preciso calcular a distancia do celular encontrado para a coordenada do arquivo do professor
    
print("% sim: " + str(sim/(sim+nao)) + " " + str(sim))
print("% nao: " + str(nao/(sim+nao)) + " " + str(nao))
print("% sim com distancia < 0.05: " + str(simComDistancia/(sim+nao)) + " " + str(simComDistancia))

  
  
  

  
        

SIM 0.008236238100781145 xEst 0.838634192943573 yEst 0.13318689167499542 xCorr 0.8306 yCorr 0.135/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/0.jpg
SIM 0.0018722405335275982 xEst 0.8732722401618958 yEst 0.17180117964744568 xCorr 0.8714 yCorr 0.1718/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/1.jpg
SIM 0.0019495337930192068 xEst 0.6855655312538147 yEst 0.6913551092147827 xCorr 0.6857 yCorr 0.6933/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/3.jpg
SIM 0.0028911111494561055 xEst 0.6680004596710205 yEst 0.6114324927330017 xCorr 0.6653 yCorr 0.6104/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/4.jpg
SIM 0.007941364859697203 xEst 0.8487153053283691 yEst 0.2194311022758484 xCorr 0.8469 yCorr 0.2117/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/5.jpg
SIM 0.004530791455405773 xEst 0.35268986225128174 yEst 0.7745878100395203 xCorr 0.3531 yCorr 0.7791/Users/henriquebueno/DOUTORADO/2018.01

SIM 0.004430277886488284 xEst 0.5392243266105652 yEst 0.28467321395874023 xCorr 0.5429 yCorr 0.2822/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/52.jpg
SIM 0.0035996132787553727 xEst 0.33166536688804626 yEst 0.07476396858692169 xCorr 0.3347 yCorr 0.0767/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/53.jpg
SIM 0.0035216975430436004 xEst 0.6869686245918274 yEst 0.7471891045570374 xCorr 0.6837 yCorr 0.7485/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/54.jpg
SIM 0.006727007072667836 xEst 0.3194776773452759 yEst 0.4707753658294678 xCorr 0.3245 yCorr 0.4663/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/55.jpg
SIM 0.005127547265003137 xEst 0.8806588053703308 yEst 0.6217629313468933 xCorr 0.8857 yCorr 0.6227/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/57.jpg
SIM 0.0019609485890356437 xEst 0.8450738787651062 yEst 0.15835322439670563 xCorr 0.8449 yCorr 0.1564/Users/henriquebueno/DOUTORADO

SIM 0.0030023899890869623 xEst 0.45642244815826416 yEst 0.628495454788208 xCorr 0.4551 yCorr 0.6258/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/102.jpg
SIM 0.0011115037070240832 xEst 0.2479453682899475 yEst 0.7695490121841431 xCorr 0.249 yCorr 0.7699/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/103.jpg
SIM 0.004303049625892469 xEst 0.7354874610900879 yEst 0.257346510887146 xCorr 0.7388 yCorr 0.2546/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/104.jpg
SIM 0.00353831010641789 xEst 0.5798142552375793 yEst 0.8408401608467102 xCorr 0.5776 yCorr 0.8436/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/105.jpg
SIM 0.002526194425145531 xEst 0.6714823842048645 yEst 0.37896984815597534 xCorr 0.6694 yCorr 0.3804/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data/106.jpg
SIM 0.0026015909702301024 xEst 0.23082639276981354 yEst 0.474204957485199 xCorr 0.2327 yCorr 0.4724/Users/henriquebueno/DOUTORADO/

In [16]:
print("acabou")

acabou
